# How to use function calling with LLMs
Function calling extends the power capabilities of LLMs. It allolws you to format
the output of an LLM response into a JSON object, which then can be fed down stream
to an actual function as a argument to process the response.

OpenAI documention states the basic steps involved in function calling: 

1. Call the model with the user query and a set of functions defined in the functions parameter.
2. The model can choose to call one or more functions; if so, the content will be a stringified JSON object adhering to your custom schema (note: the model may hallucinate parameters).
3. Parse the string into JSON in your code, and call your function with the provided arguments if they exist.
4. Call the model again by appending the function response as a new message, and let the model summarize the results back to the user.

Let's first demonstrate how we use this feature. Let's first specify a function and use the API to generate function arguments.

## How to generate function arguments


In [1]:
import warnings
import os

import openai
from openai import OpenAI

from dotenv import load_dotenv, find_dotenv

Load our .env file with respective API keys and base url endpoints. Here you can either use OpenAI or Anyscale Endpoints. **Note**: Function calling for Anyscale Endpoints is coming soon. Not yet ready as of writing this notebook!

In [2]:
_ = load_dotenv(find_dotenv()) # read local .env file
warnings.filterwarnings('ignore')
openai.api_base = os.getenv("ANYSCALE_API_BASE", os.getenv("OPENAI_API_BASE"))
openai.api_key = os.getenv("ANYSCALE_API_KEY", os.getenv("OPENAI_API_KEY"))
MODEL = os.getenv("MODEL")
print(f"Using MODEL={MODEL}; base={openai.api_base}")

Using MODEL=gpt-4-1106-preview; base=https://api.openai.com/v1


Define some utitilty functions

In [3]:
from openai import OpenAI

client = OpenAI(
    api_key = openai.api_key,
    base_url = openai.api_base
)

In [4]:
def get_commpletion(clnt: object, model: str, system_content: str, user_content:str) -> str:
    chat_completion = clnt.chat.completions.create(
        model=model,
    messages=[{"role": "system", "content": system_content},
              {"role": "user", "content": user_content}],
    temperature = 0.8)

    response = chat_completion.choices[0].message.content
    return response

Define the tools argument and function specification as part of the message


In [16]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "sum_prime_numbers",
            "description": "Compute the list of prime numbers",
            "parameters": {
                "type": "object",
                "properties": {
                    "numbers": {
                        "type": "list",
                        "description": "list of ten integer prime numbers between 2 and 100",
                    },
                    "format": {
                        "type": "list",
                        "description": "list of ten integer prime numbers between 2 and 100",
                    },
                },
                "required": ["numbers", "format"],
            },
        }
    },
]

In [17]:
system_content = """Don't make assumptions about what values to plug into functions. 
                    If the request is ambigous ask for clarification.
                 """
user_content = """Generate a list of ten random prime numbers between 2 and 100. Do generate 
Python code. Instead return a list of 10 random prime numbers and put them in a list."""

In [18]:
print(f"Using Endpoints: {openai.api_base} ...\n")
response = get_commpletion(client, MODEL, system_content, user_content)
print(response)

Using Endpoints: https://api.openai.com/v1 ...

Certainly! Here is a list of ten random prime numbers between 2 and 100:

[3, 17, 41, 67, 73, 89, 37, 23, 59, 97]
